<a href="https://colab.research.google.com/github/eeseohyun/project/blob/main/AI_05_%EC%9D%B4%EC%84%9C%ED%98%84_Section2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PROJECT2
#: 데이터셋을 이용하여 만든 머신러닝 예측 모델을 통한 성능 및 인사이트 도출 및 공유

**1) 데이터 선정 이유 및 문제 정의**

**2) 데이터를 이용한 가설 및 평가지표, 베이스라인 선택**
   1. ```해당 특성을 타겟으로 지정한 이유```
   2. ```해당 베이스라인 모델과 평가지표를 선택한 이유```

**3) EDA와 데이터 전처리**
1. EDA
2. Feature Engineering
3. 데이터의 정규화
4. 노이즈 제거
5. 결측치 제거 혹은 대체
6. 데이터 밸런스
7. 그 외

**4) 머신러닝 방식 적용 및 교차검증**

*   Linear / Tree-based / Ensemble 모델 학습
*   평가지표 계산 및 베이스라인과의 비교
*   교차 검증(이하 CV)을 통한 일반화 가능성 확인
*   모델 성능을 개선하기 위한 다양한 방법 적용
*   ```Hyperparameter tuning, etc.```

 최소 2개 이상의 모델을 만들어서 validation 점수 보고
*   최종 모델의 test 점수

**5) 머신러닝 모델 해석**

*   PDP, SHAP을 활용한 최종 모델 설명



In [ ]:
#(1)data 선정 및 문제 정의

In [ ]:
import requests
api_key = 'RGAPI-0ab7515f-5af3-4f9c-a62f-e73cdc7b40e9'
url = 'https://kr.api.riotgames.com/lol/summoner/v4/summoners/by-name/' + '진리진영' +'?api_key=' + api_key
r = requests.get(url)
r.json()['id']

In [ ]:
#소환사 랭크정보 수집
tier_url = 'https://kr.api.riotgames.com/lol/league/v4/entries/by-summoner/'+ r.json()['id'] +'?api_key=' + api_key
r2  = requests.get(tier_url)
r2.json()

소환사 랭크 정보 수집 결과
*   소환사 이름명 : 진리진영


> 솔로랭크 5:5

*   랭크(티어) : 실버4
*   랭크점수(lp) : 53점
*   승리 : 130회
*   패배 : 152회

> 자유랭크

*   랭크(티어) : 브론즈2
*   랭크점수(lp) : 29점
*   승리 : 6회
*   패배 : 6회











In [ ]:
silver_four = 'https://kr.api.riotgames.com/lol/league-exp/v4/entries/RANKED_SOLO_5x5/SILVER/IV?api_key=' + api_key
r = requests.get(silver_four) #silver4 데이터 호출
league_df = pd.DataFrame(r.json())
league_df.head()

문제점

     : 승패에 대한 예측 모델을 작성하기 위해서는 경기가 끝난 정보들을 이용할 수 없다고 봄. 따라서 경기가 시작한 후 10분 경과된 데이터를 분석한 후 예측모델을 작성하려고 함


In [ ]:
!pip install plotly==4.10.0
!pip install shap
!pip install lightgbm
!pip install bayesian-optimization

In [ ]:
import numpy as np 
import pandas as pd
import plotly as py
import plotly.express as px
import plotly.graph_objs as go
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import lightgbm as lgb
from bayes_opt import BayesianOptimization
from sklearn.metrics import roc_auc_score
import shap
import warnings  
warnings.filterwarnings('ignore')

In [ ]:
# 데이터셋 다시 불러오기 (출처 : kaggle)
import pandas as pd

data = '/content/high_diamond_ranked_10min 2.csv'
df = pd.read_csv(data)

In [ ]:
# column 확인
print(df.info())

In [ ]:
# data 확인
df.head()

**Blue VS Red Team** 

1. Game Rule


> 단계적으로 상대팀의 각 라인에 있는 타워들과 억제기를 파괴함으로써 상대팀의 넥서스를 먼저 파괴하는 팀이 우승하게 된다.





In [ ]:
color_discrete_map = {'Blue': 'rgb(122, 148, 231)', 'Red': 'rgb(255, 105, 97)'}


fig = px.histogram(df, x="blueWins", color = 'blueWins', color_discrete_map = color_discrete_map,
                  labels={"blueWins": "Teams","count": "Wins"
                 },
                title="Total Wins per Team",
                hover_name="blueWins",       
                  )

py.offline.iplot(fig)

블루팀과 레드팀의 승패는 거의 동일하다.

2. Wards (와드/시야)


> 지도의 특정 지역에서 안개를 제거하여 시야를 밝히는 것으로 구매 아이템을 통해 부여받을 수 있다.



In [ ]:
color_discrete_map = {'Blue': 'rgb(122, 148, 231)', 'Red': 'rgb(255, 105, 97)'}

layout = go.Layout(
    yaxis=dict(
        range=[5, 45]
    ),
    xaxis=dict(
        range=[100, 200]
    )
)

tmp1 = df[['blueWardsPlaced', 'blueWardsDestroyed']].copy()
tmp1.columns = ['WardsPlaced','WardsDestroyed']
tmp1 = tmp1.astype(float)
tmp1['Team'] = 'Blue'
tmp2 = df[['redWardsPlaced', 'redWardsDestroyed']].copy()
tmp2.columns = ['WardsPlaced','WardsDestroyed']
tmp2 = tmp2.astype(float)
tmp2['Team'] = 'Red'

data2 = pd.concat([tmp1, tmp2], ignore_index = True).sample(2000)
data2

In [ ]:
fig1 = px.violin(data2, y="WardsPlaced", color = 'Team',  box=True, points='all', color_discrete_map=color_discrete_map, title = 'Wards Placed per Team')

fig1.show()

양쪽팀 대부분 게임 10분 동안 10개 ~ 25개 와드를 설치한다.

배치된 와드 수의 분포는 비슷하다.

2-1. Wards Destroyed



In [ ]:
fig1 = px.violin(data2, y="WardsDestroyed", color = 'Team',  box=True, points='all', color_discrete_map=color_discrete_map, title = 'Wards Destroyed per Team')

fig1.show()

평균적으로 2개 와드가 파괴되며, 주로 0과 6 사이에 분포하고 있다.

3. First Bloods

> 게임의 첫 챔피언 킬



In [ ]:
color_discrete_map = {'Blue': 'rgb(122, 148, 231)', 'Red': 'rgb(255, 105, 97)'}

tmp3 = df[['blueFirstBlood']].copy()
tmp3.columns = ['FirstBloods']
tmp3 = tmp3.astype(float)
tmp3['Team'] = 'Blue'
tmp4 = df[['redFirstBlood']].copy()
tmp4.columns = ['FirstBloods']
tmp4 = tmp4.astype(float)
tmp4['Team'] = 'Red'
data3 = pd.concat([tmp3, tmp4])
data3 = data3.groupby('Team').mean().reset_index()

data3

상대적으로 양쪽이 동일함

4. Kill, Deaths and Assists (KDA)

*   Kill : 상대 챔피언을 죽인 횟수
*   Deaths : 상대 챔피언에게 죽은 횟수
*   Assists : 상대 챔피언을 죽이도록 도와준 횟수






In [ ]:
col = ['blueKills','blueDeaths','blueAssists','redKills','redDeaths','redAssists']

tmp5 = df[col[0:3]].copy()
tmp5.columns = ['Kills','Deaths','Assists']
tmp5['Team'] = 'Blue'
tmp6 = df[col[3:6]].copy()
tmp6.columns = ['Kills','Deaths','Assists']
tmp6['Team'] = 'Red'

data4 = pd.concat([tmp5, tmp6], ignore_index = True)
data4 = data4.groupby('Team').mean().reset_index()
data4 = pd.melt(data4, id_vars=['Team'], value_vars=['Kills','Deaths','Assists'])
data4.columns = ['Team','KDA','Mean']

data4

In [ ]:
fig = px.bar(data4, x="Team", y="Mean", color="Team", color_discrete_map=color_discrete_map,
             facet_col="KDA", title = 'Mean KDA per Team')

            
fig.show()

*   kills : 레드팀 < 블루팀
*   deaths : 레드팀 > 블루팀
*   assists : 레드팀 > 블루팀






5. Neutral Monsters


> 양 팀 모두에게 공격받을 수 있고, 챔피언에게 공격 받기 전까진 선공하지 않는다. 공격로 미니언보다 능력치가 훨씬 높아 처치하기 까다롭지만 그 만큼 많은 골드와 경험치, 버프를 제공해준다.



In [ ]:
col = ['blueEliteMonsters','blueDragons','blueHeralds','redEliteMonsters','redDragons','redHeralds']
tmp7 = df[col[0:3]].copy()
tmp7.columns = ['EliteMonsters','Dragons','Heralds']
tmp7['Team'] = 'Blue'
tmp8 = df[col[3:6]].copy()
tmp8.columns = ['EliteMonsters','Dragons','Heralds']
tmp8['Team'] = 'Red'

data5 = pd.concat([tmp7, tmp8], ignore_index = True)
data5 = data5.groupby('Team').mean().reset_index()
data5 = pd.melt(data5, id_vars=['Team'], value_vars=['EliteMonsters','Dragons','Heralds'])
data5.columns = ['Team','NeutralGoals','Mean']
data5

In [ ]:
fig = px.bar(data5, x="Team", y="Mean", color="Team", color_discrete_map=color_discrete_map,
             facet_col="NeutralGoals", title = 'Mean Neutral Goals per Team'
            )
fig.show()

레드팀은 EliteMonster와 Dragon, 블루팀은 전령을 더 많이 잡는다.

6. Turrets(포탑/타워)

> 상대 챔피언에게 피해를 입히고 그들의 팀에 시야를 제공하여 전장을 더 잘 통제할 수 있게 하며 한 번에 한 유닛을 겨냥해 큰 피해를 입힌다. 각 팀은 적진으로 돌격하기 위해 적 포탑을 파괴해야 합니다.



In [ ]:
col = ['blueTowersDestroyed', 'redTowersDestroyed']
tmp9 = df[col[0:1]].copy()
tmp9.columns = ['TowersDestroyed']
tmp9['Team'] = 'Blue'
tmp10 = df[col[1:2]].copy()
tmp10.columns = ['TowersDestroyed']
tmp10['Team'] = 'Red'

data6 = pd.concat([tmp9, tmp10], ignore_index = True)
data6 = data6.groupby('Team').mean().reset_index()
data6

In [ ]:
fig = px.bar(data6, x="Team", y="TowersDestroyed", color="Team", color_discrete_map=color_discrete_map,title = 'Mean Turrets Destroyed per Team'
            )
fig.show()

블루팀이 레드팀보다 훨씬 더 많은 포탑을 파괴하였고, 거의 100개 가량 더 파괴한다.

이는 레드팀의 정글몹의 집중이라고 예상할 수 있다.

7. Golds


> 챔피언에게 보너스 통계와 능력을 제공하는 아이템을 구입하는 데 사용되며, 이는 챔피언이 경기 진행 중에 힘을 키우는 주력이다.




In [ ]:
col = ['blueTotalGold', 'redTotalGold']
tmp11 = df[col[0:1]].copy()
tmp11.columns = ['TotalGold']
tmp11['Team'] = 'Blue'
tmp12 = df[col[1:2]].copy()
tmp12.columns = ['TotalGold']
tmp12['Team'] = 'Red'

data7 = pd.concat([tmp11, tmp12], ignore_index = True)
data7 = data7.groupby('Team').mean().reset_index()
data7

In [ ]:
fig = px.bar(data7, x="Team", y="TotalGold", color="Team", color_discrete_map=color_discrete_map,title = 'Mean Gold per Team'
            )
fig.show()

8. Leveling and Experiance


> 챔피언 경험(XP)은 챔피언이 일정 경력에 도달하면 레벨업할 수 있는 게임 메카닉이다. 레벨업을 통해 새로운 능력이나 기존 능력의 상위 등급에 접근할 수 있습니다. 



In [ ]:
#Leveling
col = ['blueAvgLevel', 'redAvgLevel']
tmp13 = df[col[0:1]].copy()
tmp13.columns = ['AvgLevel']
tmp13['Team'] = 'Blue'
tmp14 = df[col[1:2]].copy()
tmp14.columns = ['AvgLevel']
tmp14['Team'] = 'Red'

data8 = pd.concat([tmp13, tmp14], ignore_index = True)
data8 = data8.groupby('Team').mean().reset_index()
data8

In [ ]:
#Experience
col = ['blueTotalExperience', 'redTotalExperience']
tmp15 = df[col[0:1]].copy()
tmp15.columns = ['TotalExperience']
tmp15['Team'] = 'Blue'
tmp16 = df[col[1:2]].copy()
tmp16.columns = ['TotalExperience']
tmp16['Team'] = 'Red'

data9 = pd.concat([tmp15, tmp16], ignore_index = True)
data9 = data9.groupby('Team').mean().reset_index()
data9

9. Minions Farme 


**Winning Condition** 

 1) Blue Team Heatmap 

In [ ]:
corr = df[[col for col in df.columns if 'blue' in col and col != 'blueWins']].corr()
f,ax = plt.subplots(figsize=(10, 10))
p = sns.heatmap(corr,
                cmap='coolwarm',
                annot=True,
                fmt=".1f",
                annot_kws={'size':10},
                cbar=False,
                ax=ax)
p.set_title('Blue Team Features Correlation')

1. Positive correlation between total gold and gold per minute, gold diff, total experience, assists and kills. and correlate negatively with Deaths.

2. Positive correlation between elite monsters and dragons.

3. Positive correlation between golds per minute and kills.

4. negative correlation between deaths and gold diff, experience diff.

2) Red Team Heatmap

In [ ]:
corr = df[[col for col in df.columns if 'red' in col and col != 'blueWins']].corr()
f,ax = plt.subplots(figsize=(10, 10))
p = sns.heatmap(corr,
                cmap='coolwarm',
                annot=True,
                fmt=".1f",
                annot_kws={'size':10},
                cbar=False,
                ax=ax)

3) 승리와 패배의 상관관계

In [ ]:
df['blueWins']

In [ ]:
blue_win = df[[col for col in df.columns if col != 'blueWins']].corrwith(df['blueWins']).to_frame().sort_values(by = 0, ascending = False)
blue_win = pd.concat([blue_win.head(5), blue_win.tail(5)])
blue_win.columns = ['Blue Win Correlation']
blue_win

In [ ]:
red_win = df[[col for col in df.columns if col != 'blueWins']].corrwith(df['blueWins'].map({0:1, 1:0})).to_frame().sort_values(by = 0, ascending = False)
red_win = pd.concat([red_win.head(5), red_win.tail(5)])
red_win.columns = ['Red Win Correlation']
red_win

In [ ]:
fig = plt.figure(figsize=(25,10))
ax1 = fig.add_subplot(121)
ax2 = fig.add_subplot(122)

plt.figure(figsize=(6,6))

sns.heatmap(blue_win,
            vmin=-1,
            cmap='coolwarm',
            annot=True,
           ax = ax1);

sns.heatmap(red_win,
            vmin=-1,
            cmap='coolwarm',
            annot=True,
           ax = ax2);

경험치나 골드에서 한 팀이 훨씬 더 많은 것이 승리할 가능성이 높다.

**Modeling**

In [ ]:
X = df.drop(['blueWins', 'gameId'], axis=1)
y = df.blueWins

In [ ]:
def bayes_parameter_opt_lgb(X = X, y = y, init_round=15, opt_round=25, n_folds=5, random_seed=6, n_estimators=10000, learning_rate=0.05, output_process=False):
    # prepare data
    train_data = lgb.Dataset(data=X, label=y, free_raw_data=False)
    # parameters
    def lgb_eval(num_leaves, feature_fraction, bagging_fraction, max_depth, lambda_l1, lambda_l2, min_split_gain, min_child_weight):
        params = {'application':'binary','num_iterations': n_estimators, 'learning_rate':learning_rate, 'early_stopping_round':100, 'metric':'auc'}
        params["num_leaves"] = int(round(num_leaves))
        params['feature_fraction'] = max(min(feature_fraction, 1), 0)
        params['bagging_fraction'] = max(min(bagging_fraction, 1), 0)
        params['max_depth'] = int(round(max_depth))
        params['lambda_l1'] = max(lambda_l1, 0)
        params['lambda_l2'] = max(lambda_l2, 0)
        params['min_split_gain'] = min_split_gain
        params['min_child_weight'] = min_child_weight
        cv_result = lgb.cv(params, train_data, nfold=n_folds, seed=random_seed, stratified=True, verbose_eval =200, metrics=['auc'])
        return max(cv_result['auc-mean'])    
    # range 
    lgbBO = BayesianOptimization(lgb_eval, {'num_leaves': (24, 45),
                                            'feature_fraction': (0.1, 0.9),
                                            'bagging_fraction': (0.8, 1),
                                            'max_depth': (5, 8.99),
                                            'lambda_l1': (0, 5),
                                            'lambda_l2': (0, 3),
                                            'min_split_gain': (0.001, 0.1),
                                            'min_child_weight': (5, 50)}, random_state=0)
    # optimize
    lgbBO.maximize(init_points=init_round, n_iter=opt_round)
    
    # output optimization process
    if output_process==True: lgbBO.points_to_csv("bayes_opt_result.csv")
    
    # return best parameters
    return lgbBO

opt_params = bayes_parameter_opt_lgb(X, y, init_round=5, opt_round=10, n_folds=3, random_seed=6, n_estimators=100, learning_rate=0.05)

In [ ]:
params = opt_params.max['params']
params["num_leaves"] = int(round(params['num_leaves']))
params['feature_fraction'] = max(min(params['feature_fraction'], 1), 0)
params['bagging_fraction'] = max(min(params['bagging_fraction'], 1), 0)
params['max_depth'] = int(round(params['max_depth']))
params['lambda_l1'] = max(params['lambda_l1'], 0)
params['lambda_l2'] = max(params['lambda_l2'], 0)
params['min_split_gain'] = max(params['min_split_gain'], 0)
params['min_child_weight'] = max(params['min_child_weight'], 0)
params['num_iterations'] = int(round(10000))
params['learning_rate'] = max(0.05, 0)
params['application'] = 'binary'
params['metric'] = 'auc'
params['objective'] = 'binary'

In [ ]:
train_data = lgb.Dataset(data=X, label=y, free_raw_data=True, feature_name = [col for col in df.columns if col!= 'gameId' and col!= 'blueWins'])
model = lgb.train(params, train_data)

In [ ]:
ax = lgb.plot_importance(model, max_num_features=10, figsize = (15,15))
plt.show()

가장 중요한 특징은 총 경험치, 총 골드량, 그리고 처치한 미니언들이다.

**모델이 실제 차이와 비교했을 때 어떻게 승패를 구별할 수 있는지 비교**

3D 산란을 이용한 플롯팅 

```파란색 : 승,  빨간색 : 패```

In [ ]:
pred = np.round(model.predict(X)).astype(int)
data_test = TSNE(n_components=3).fit_transform(X)

In [ ]:
predicted = pd.DataFrame(data_test)
predicted['output'] = pred
predicted['output'] = predicted['output'].astype(object)
predicted['output'] = predicted['output'].map({0: "Red Wins",1: "Blue Wins"})
predicted['flag'] = 'predicted'
predicted.columns = ['dim_1','dim_2','dim_3','output', 'flag']

real = pd.DataFrame(data_test)
real['output'] = y.values
real['output'] = real['output'].astype(object)
real['output'] = real['output'].map({0: "Red Wins",1: "Blue Wins"})
real['flag'] = 'real'
real.columns = ['dim_1','dim_2','dim_3','output', 'flag']

data_3d = pd.concat([predicted,real], ignore_index = True).sample(4000)

In [ ]:
color_discrete_map = {"Blue Wins": 'rgb(122, 148, 231)', "Red Wins": 'rgb(255, 105, 97)'}

fig1 = px.scatter_3d(predicted, x='dim_1', y='dim_2', z='dim_3', color_discrete_map=color_discrete_map,color='output', title = 'How Model Distinct Wins and Loses')
fig2 = px.scatter_3d(real, x='dim_1', y='dim_2', z='dim_3',color_discrete_map=color_discrete_map,color='output', title = 'Real Distinction Between Wins and Loses')

fig1.show()
fig2.show()

포인트가 약간 얽혀있지만 한쪽이 더 파란색이고 다른 한쪽은 빨간색이 더 많이 섞여있는 것을 볼 수 있다.


**결론**

양팀 모두 상당히 균등하지만 그중에서도 블루팀은 더 많은 골드와 킬의 혜택을 받으며 레드팀은 더 많은 정글몹과 경험치, CS의 혜택을 받는다.